In [3]:
from database.TradeHistory import get_session, History
import pandas as pd

In [19]:
df = pd.read_sql_query("select * from history where exec_date>='2018-01-11 14:15:00'", session.bind)
df.exec_date = pd.to_datetime(df.exec_date)
df = df.set_index('exec_date')
df = df.loc['2016-03':]
df = df[['id', 'price', 'size']]
bench_price = df.price.resample('15Min').mean().fillna(method='ffill')
bench_size = df['size'].resample('15Min').sum().fillna(0)
dfb = pd.DataFrame([bench_price, bench_size]).T

In [20]:
dfb

,price,size
exec_date,,
2018-01-11 14:15:00,1.748889e+06,97.615653


In [21]:
dfb.to_sql('history15min', session.bind, chunksize=1000, if_exists='append')

IntegrityError: (mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry '2018-01-11 14:15:00' for key 'PRIMARY' [SQL: 'INSERT INTO history15min (exec_date, price, size) VALUES (%(exec_date)s, %(price)s, %(size)s)'] [parameters: {'exec_date': datetime.datetime(2018, 1, 11, 14, 15), 'price': 1748888.9422521656, 'size': 97.61565295000032}]

In [17]:
df.tail(5)

,id,price,size
exec_date,,,
2018-01-11 14:22:16,111342754,1744900,0.010
2018-01-11 14:22:17,111342758,1744900,0.030
2018-01-11 14:22:21,111342800,1744900,0.059
2018-01-11 14:22:21,111342804,1744900,0.010
2018-01-11 14:22:21,111342807,1744900,0.001
